In [44]:
!git clone https://github.com/rsuwaileh/IDRISI.git

fatal: destination path 'IDRISI' already exists and is not an empty directory.


In [45]:
pip install 'scikit-learn<0.24' sklearn-crfsuite scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import os
import csv
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import json
import requests

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [47]:
def tweet_read(path):
    tokens = []
    labels = []
    t = []
    l = []
    for token in open(path, encoding='utf-8').read().splitlines(): 
        if token == '':
            tokens.append(t)
            labels.append(l)
            t = []
            l = []
            continue
        splits = token.split()
        t.append(splits[0])
        l.append(splits[1])

    if len(t) > 0 and len(l) > 0:
        tokens.append(t)
        labels.append(l)
        
    return tokens, labels


def get_pos(tokens):
    pos = []
    for i in range(0, len(tokens)):
        t_pos = []


        for e in nltk.pos_tag(tokens[i]):


            for p in e[1].split("\n"):
                t_pos.append(p)
        if len(t_pos) != len(tokens[i]):
            print("mismatch in length")
        pos.append(t_pos)
    return pos

def conver_to_crf(tokens, pos, labels, out_path):
    with open(out_path, mode='w', encoding="utf-8", newline="") as data_file:
        writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Sentence #","Word","POS","Tag"])
        for tweet in range(0, len(tokens)):
            begin = True
            for j in range(0, len(tokens[tweet])):
                if begin:
                    writer.writerow(["Sentence: " + str(tweet+1), tokens[tweet][j], pos[tweet][j], labels[tweet][j]])
                    begin = False
                else:
                    writer.writerow(["", tokens[tweet][j], pos[tweet][j], labels[tweet][j]])
def convert_to_typeless(labels):
    tllabels = []
    
    for i in range(len(labels)):
        tllabels.append([])
        for l in labels[i]:
            spls = l.split("-")
            if len(spls) > 1:
                tllabels[i].append(spls[0]+"-LOC")
            else:
                tllabels[i].append(l)
    
    return tllabels 

In [48]:
path = "/content/" + "IDRISI/LMR/data/"
disaster = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

for typ in ['typeless']:
    for case in ['random']:
        for event in disaster:

            in_path = path + "EN/gold-" + case + "-bilou/"+ event 
            out_path = path + "EN/gold-" + case + "-bilou-crf/" + typ + "/" + event 
            if not os.path.exists(out_path):
                os.makedirs(out_path)

            for prt in ["train", "dev"]:  
                prt_in_path = in_path + "/" + prt + ".txt"
                tokens, labels = tweet_read(prt_in_path)
                pos = get_pos(tokens)
                if typ == 'typeless':
                    labels = convert_to_typeless(labels)
                prt_out_path = out_path + "/" + prt + ".csv"
                conver_to_crf(tokens, pos, labels, prt_out_path)

In [49]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        #print('self.grouped')
        #print(len(self.grouped))
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try:
            s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s["Word"].values.tolist(), s["POS"].values.tolist(), s["Tag"].values.tolist()    
        except:
            self.empty = True
            return None, None, None
    
        def get_next(self):
            try:
                s = self.grouped["Sentence: {}".format(self.n_sent)]
                self.n_sent += 1
                return s
            except:
                return None


In [50]:
import pandas as pd
import numpy as np
# import SentenceGetter as SG
from sklearn_crfsuite import CRF
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

import scipy.stats
import nltk
import sklearn
from sklearn.metrics import make_scorer

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [17]:
def read_data(file_name):
    data = pd.read_csv(file_name, encoding="utf-8")
    data = data.fillna(method="ffill")
    words = list(set(data["Word"].values))
    tags = list(set(data["Tag"].values))

    # getter = SG.SentenceGetter(data)
    getter = SentenceGetter(data)

    sent, pos, tag = getter.get_next()
    sentences = getter.sentences
    return sentences, words, tags

In [51]:
# 2. Feature preparation
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [52]:
def train_crf(X, y, cv, algorithm, c1, c2, max_iterations):
    crf = CRF(
        algorithm='lbfgs',
        max_iterations=100,
        all_possible_transitions=True
    )
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
    }

    labels = ['B-LOC', 'L-LOC', 'U-LOC', 'I-LOC']

    # use the same metric for evaluation
    f1_scorer = make_scorer(metrics.flat_f1_score,
                            average='weighted', labels=labels)

    # search
    rs = RandomizedSearchCV(crf, params_space,
                            cv=3,
                            verbose=1,
                            n_jobs=-1,
                            n_iter=50,
                            scoring=f1_scorer)
    rs.fit(X, y)
    return rs


    
def run_predict(crf, X_test):   
    return crf.predict(X_test)

In [53]:
def dump_predictions(output_path, tokens, labels):
    writer = open(output_path, 'w', encoding='utf-8', newline="")
    
    for i in range(len(labels)):
        t = [x for x, y, z in tokens[i]]
        for j in range(len(labels[i])):
            writer.write(t[j] + "\t" + labels[i][j] + "\n")
        writer.write("\n")

In [54]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import matplotlib.pyplot as plt

In [56]:
# Driver Code
def run(event, train_path, test_path, out_path):
    
    sentences_tr, words_tr, tags_tr = read_data(train_path)
    sentences_te, words_te, tags_te = read_data(test_path)

    X_train = [sent2features(s) for s in sentences_tr] 
    y_train = [sent2labels(s) for s in sentences_tr] 
    X_test = [sent2features(s) for s in sentences_te]
    y_test = [sent2labels(s) for s in sentences_te]

    crf = train_crf(X_train, y_train, 5, 'lbfgs', 0, 1, 100)

    y_pred = crf.predict(X_test)
    y_pred = cross_val_predict(estimator=crf, X=X_test, y=y_test, cv=5)

    labels = list(crf.classes_)
    labels.remove('O')
    print(labels)

    sorted_labels = sorted(
        labels,
        key=lambda name: (name[1:], name[0])
    )
    scores = metrics.flat_classification_report(
        y_test, y_pred, labels=sorted_labels, digits=3
    )
            
    dump_predictions(out_path, sentences_te, y_pred)

    return X_train, y_train, scores

In [58]:
%%time
# path = "<path to IDRISI data directory>\\IDRISI\\data\\LMR\\"
path = "/content/" + "IDRISI/LMR/data/"

events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

  
for typ in ['typeless']:# 'typefull', 
    for case in ['random']: # , 'timebased'
        for event in events:
            in_path = path + "EN/gold-" + case + "-bilou-crf/" + typ + "/" + event 
            train_path = in_path + "/train.csv"
            test_path = in_path + "/dev.csv"
            out_path = path + "EN/gold-" + case + "-bilou-crf/" + typ + "/" + event + "-predictions.txt"
            run(event, train_path, test_path, out_path) 



            X_train, y_train, scores = run(event, train_path, test_path, out_path)  
            break # Remove this

ValueError: ignored

In [43]:
%%time
print(scores)

              precision    recall  f1-score   support

       B-LOC      0.727     0.421     0.533        19
       I-LOC      0.000     0.000     0.000         1
       L-LOC      0.727     0.421     0.533        19
       U-LOC      0.902     0.783     0.838       106

   micro avg      0.868     0.683     0.764       145
   macro avg      0.589     0.406     0.476       145
weighted avg      0.850     0.683     0.753       145

CPU times: user 955 µs, sys: 0 ns, total: 955 µs
Wall time: 1.07 ms
